<style>
pre > code {
    background-color: #3A3960 !important;
    padding: 10px;
    display: block;
    border-radius: 5px;
    border: 1px solid #ccc;
    overflow-x: auto;
}
</style>

# Python requests-Modul im Detail

Nun werden wir das requests-Modul Schritt für Schritt im Detail untersuchen. Dabei werden wir die API von httpbin.org nutzen, um verschiedene HTTP-Methoden zu testen.
Httpbin ist eine öffentliche Test-API, die speziell entwickelt wurde, um HTTP-Anfragen zu debuggen und zu verstehen.

## Das Response-Objekt

Wir hatten bereits gesehen wie einfach es ist, eine GET-Request zu erstellen:

```python
import requests
import json

BASE_URL = "https://httpbin.org/"
ENDPOINT_1 = "/get"

response = requests.get(BASE_URL + ENDPOINT_1)
print(json.dumps(response.json(), indent=4))
```

Das response-Objekt ist das Herzstück einer HTTP-Anfrage mit requests. Es enthält alle wichtigen Informationen über die Antwort, die der Server zurückgegeben hat.
Das response-Objekt hat viele wichtige Attribute und Methoden, in unserem Beispiel haben wir die "json()" Methode verwendet. Wenn die Antwort ein gültiges JSON-Dokument ist, kann man es direkt mit dieser Methode, in ein Python-Dictionary umwandeln. Damit dieses Python-Dictionary jedoch lesbar ist, wandeln wir es während der Konsolenausgabe zurück in eine lesbare JSON-Formatierung um, mit einer Einrückung von 4 Leerzeichen. So sieht dann die Konsolenausgabe aus:

```
{
    "args": {},
    "headers": {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate",
        "Host": "httpbin.org",
        "User-Agent": "python-requests/2.32.3",
        "X-Amzn-Trace-Id": "Root=1-679f3457-68a6eaa93609447f594d8f06"
    },
    "origin": "146.52.27.9",
    "url": "https://httpbin.org/get"
}
```

Man muss beachten dass nicht jede API eine JSON-Antwort gibt, auch wenn es oft als Standardformat für APIsbezeichnet wird. Deswegen sollte man immer mit Exceptions arbeiten, da ein "JSONDecodeError" vorkommen könnte:

```python
response = requests.get(BASE_URL + ENDPOINT_1)
try:
    data = response.json()
    print(json.dumps(data, indent=4))
except json.JSONDecodeError:
    print("Fehler: Antwort ist kein gültiges JSON!")
```

Mit dem "text"-Attribut, kann man sich die API-Response unformatiert als reinen Text ausgeben lassen. Eventuell für HTML-Daten notwendig, hier liegt der Schwerpunkt auf HTML-Website-Data-Scraping:

```python
response = requests.get(BASE_URL + ENDPOINT_1)
try:
    data = response.text
    print(data)
    print(type(data)) # <class 'str'>
except json.JSONDecodeError:
    print("Fehler: Antwort ist kein gültiges JSON!")
```

Es kann aber auch vorkommen dass man die Rohdaten der Response benötigt. Dies kann besonders nützlich sein wenn man mit Bildern, PDFs oder Binärdateien arbeitet:

```python
response = requests.get(BASE_URL + ENDPOINT_1)
try:
    data = response.content
    print(data)
    print(type(data)) # <class 'bytes'>
except json.JSONDecodeError:
    print("Fehler: Antwort ist kein gültiges JSON!")
```

Jede HTTP-Response hat einen Statuscode, der angibt, ob die Anfrage erfolgreich war oder nicht. Über das Attribut "status_code" können wir auf die HTTP-Statuscode zugreifen:

```python
response = requests.get(BASE_URL + ENDPOINT_1)
try:
    data = response.json()
    print(json.dumps(response.json(), indent=4))
    status = response.status_code
    print(f"Statuscode: {status}") # 200
except json.JSONDecodeError:
    print("Fehler: Antwort ist kein gültiges JSON!")
```

Man soltle sich gewöhnen, den Statuscode immer vor der Verarbeitung der Daten zu überprüfen:

```python
response = requests.get(BASE_URL + ENDPOINT_1)
try:
    data = response.json()
    status = response.status_code
    print(f"Statuscode: {status}") # 200
    if status == 200:
        print("Anfrage erfolgreich!")
        print(json.dumps(response.json(), indent=4))
    else:
        print(f"Fehler mit dem Statuscode: {status}")

except json.JSONDecodeError:
    print("Fehler: Antwort ist kein gültiges JSON!")
```

Man kann jedoch den if-else-Block sich sparen und weniger Code schreiben, indem man die "raise_for_status()" Methode verwendet. Dadurch wird direkt eine Exception ausgelöst:

```python
try:
    response = requests.get(BASE_URL + ENDPOINT_1)
    response.raise_for_status()
    data = response.json()
    print(json.dumps(response.json(), indent=4))

except requests.exceptions.HTTPError as e:
    print(f"Fehler: {e}")
except json.JSONDecodeError:
    print("Fehler: Antwort ist kein gültiges JSON!")
```

Eine HTTP-Response hat immer einen HTTP-Header, diesen können wir mit dem Attribut "headers" ausgeben:  

```python
    header = dict(response.headers) # CaseInsensitiveDict in ein normales python-dict umwandeln
    print(json.dumps(header, indent=4))
```

So sieht dann der Header aus:

```
{
    "Date": "Sun, 02 Feb 2025 10:48:37 GMT",
    "Content-Type": "application/json",
    "Content-Length": "305",
    "Connection": "keep-alive",
    "Server": "gunicorn/19.9.0",
    "Access-Control-Allow-Origin": "*",
    "Access-Control-Allow-Credentials": "true"
}
```

Häufig hat man es beim Arbeiten mit APIs mit sogenannten Redirections (Weiterleitungen) kontakt. Ein Redirect (Umleitung) bedeutet, dass ein Server einer Anfrage mitteilt, dass die gewünschte Ressource nicht direkt unter der angeforderten URL verfügbar ist. Stattdessen gibt er eine neue Ziel-URL an, zu der der Client (z. B. requests) weitergeleitet werden soll. Der Ablauf sieht grob so aus:
1. Client sendet eine Anfrage (GET, POST, etc.).
2. Server antwortet mit einem Redirect-Statuscode (3xx).
3. Der Header enthält eine Location:-Angabe mit der neuen URL.
4. Client folgt automatisch (Standardverhalten in requests).
5. Neue Anfrage an die Ziel-URL wird automatisch gesendet.

Mit dem "url" Attribut lässt sich die endgültige URL der Request auslesen:

```python
import requests
import json

BASE_URL = "https://httpbin.org/"
ENDPOINT_1 = "get"
ENDPOINT_2 = "redirect/3"

try:
    response = requests.get(BASE_URL + ENDPOINT_2)
    response.raise_for_status()

    data = response.json()
    print(json.dumps(data, indent=4))

    header = dict(response.headers) # CaseInsensitiveDict in ein normales python-dict umwandeln
    print(json.dumps(header, indent=4))

    status_code = response.status_code
    print(f"Statuscode ist: {status_code}")

    url = response.url
    print(f"Endgültige URL: {url}")
    
except requests.exceptions.HTTPError as e:
    print(f"Fehler: {e}")
except json.JSONDecodeError:
    print("Fehler: Antwort ist kein gültiges JSON!")
```

Manchmal ist es interessanter zu sehen, wie und wo genau man umgeleitet wurde, anstelle nur die endgültige URL sich ausgeben zu lassen. Dazu können wir das Attribut "history" verwenden.
Mit "response.history" erhält man eine Liste aller vorherigen Response-Objekte, die zu dieser Weiterleitung geführt haben. Jedes dieser Objekte enthält Informationen über die Zwischen-Statuscodes und die neue Ziel-URL. Dadurch können wir eine detaillierte Weiterleitungsverfolgung gewährleisten:

```python
import requests
import json

BASE_URL = "https://httpbin.org/"
ENDPOINT_1 = "get"
ENDPOINT_2 = "redirect/3"

try:
    response = requests.get(BASE_URL + ENDPOINT_2)
    response.raise_for_status()

    data = response.json()
    print(json.dumps(data, indent=4))

    redirection_history = response.history
    print(redirection_history) # [<Response [302]>, <Response [302]>, <Response [302]>]

except requests.exceptions.HTTPError as e:
    print(f"Fehler: {e}")
except json.JSONDecodeError:
    print("Fehler: Antwort ist kein gültiges JSON!")
```

Mit einer for-Schleife können wir auf alle Response-Objekte der History zugreifen und wichtige informationen ausgeben:

```python
import requests
import json

BASE_URL = "https://httpbin.org/"
ENDPOINT_1 = "get"
ENDPOINT_2 = "redirect/3"

try:
    response = requests.get(BASE_URL + ENDPOINT_2)
    response.raise_for_status()

    data = response.json()
    print(json.dumps(data, indent=4))
    print(f"Endgültige URL: {response.url}")
    print(f"Endgültiger Statuscode: {response.status_code}")

    if response.history:
        print("Verlauf der Redirections:")
        for index, response in enumerate(response.history):
            print(f"\nIndex {index + 1}: Status {response.status_code}")
            print(f"Location: {response.headers.get('Location', 'Keine Weiterleitung')}")
            print(f"Header: {json.dumps(dict(response.headers), indent=4)}")
    else:
        print("Keine Weiterleitung!")
    

except requests.exceptions.HTTPError as e:
    print(f"Fehler: {e}")
except json.JSONDecodeError:
    print("Fehler: Antwort ist kein gültiges JSON!")
```

Die Ausgabe sieht dann so aus:

```
{
    "args": {},
    "headers": {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate",
        "Host": "httpbin.org",
        "User-Agent": "python-requests/2.32.3",
        "X-Amzn-Trace-Id": "Root=1-679f588a-7ff827d16dfb76613c43b4d8"
    },
    "origin": "146.52.27.9",
    "url": "https://httpbin.org/get"
}
Endgültige URL: https://httpbin.org/get
Endgültiger Statuscode: 200
Verlauf der Redirections:

Index 1: Status 302
Location: /relative-redirect/2
Header: {
    "Date": "Sun, 02 Feb 2025 11:35:37 GMT",
    "Content-Type": "text/html; charset=utf-8",
    "Content-Length": "247",
    "Connection": "keep-alive",
    "Server": "gunicorn/19.9.0",
    "Location": "/relative-redirect/2",
    "Access-Control-Allow-Origin": "*",
    "Access-Control-Allow-Credentials": "true"
}

Index 2: Status 302
Location: /relative-redirect/1
Header: {
    "Date": "Sun, 02 Feb 2025 11:35:38 GMT",
    "Content-Type": "text/html; charset=utf-8",
    "Content-Length": "0",
    "Connection": "keep-alive",
    "Server": "gunicorn/19.9.0",
    "Location": "/relative-redirect/1",
    "Access-Control-Allow-Origin": "*",
    "Access-Control-Allow-Credentials": "true"
}

Index 3: Status 302
Location: /get
Header: {
    "Date": "Sun, 02 Feb 2025 11:35:38 GMT",
    "Content-Type": "text/html; charset=utf-8",
    "Content-Length": "0",
    "Connection": "keep-alive",
    "Server": "gunicorn/19.9.0",
    "Location": "/get",
    "Access-Control-Allow-Origin": "*",
    "Access-Control-Allow-Credentials": "true"
}
```

Was genau passiert hier nun:
1. Die ursprüngliche Anfrage an "redirect/3" (https://httpbin.org/redirect/3):
   1. Die Anfrage startet bei "redirect/3", was bedeutet, dass die Antwort eine 302-Weiterleitung ist.
   2. Die Location-Header gibt an, wohin die nächste Anfrage geht.
   3. "redirect/3" bedeutet, dass es drei aufeinanderfolgende Weiterleitungen gibt.
2. Die Weiterleitungskette (response.history):
   1. "redirect/3" leitet zuerst zu "redirect/2"
   2. "redirect/2" leitet zu "redirect/1"
   3. "redirect/1" leitet zu "get"
   4. Die finale URL ist also: https://httpbin.org/get
3. Finale Antwort (200 OK):
   1. Nachdem requests alle drei Weiterleitungen befolgt hat, erhältst du am Ende eine 200 OK-Antwort von https://httpbin.org/get.
   2. Das bedeutet, dass der endgültige Inhalt von /get stammt.

## Cookies 

Cookies sind kleine Textdateien, die von einer Website oder einem Webserver an den Browser eines Nutzers gesendet und dort gespeichert werden. Sie dienen dazu, Informationen zwischen Sitzungen zu speichern, damit der Nutzer beim nächsten Besuch wiedererkannt wird.
<br>
<br>
Es gibt verschiedene Arten von Cookies, die je nach ihrem Zweck und ihrer Speicherung unterschieden werden:

1. Sitzungscookies (Session Cookies):
   1. Werden nur für die Dauer der Sitzung gespeichert und nach dem Schließen des Browsers gelöscht.
   2. Beispiel: Login-Status wird gespeichert, solange die Webseite geöffnet ist.
2. Permanente Cookies (Persistent Cookies):
   1. Bleiben über die Sitzung hinaus gespeichert (z. B. mehrere Tage, Wochen oder Monate).
   2. Beispiel: Eine Webseite merkt sich deine Spracheinstellung für den nächsten Besuch.
3. Erstanbieter-Cookies (First-Party Cookies):
   1. Werden von der besuchten Website gesetzt und nur für diese genutzt.
   2. Beispiel: Ein Online-Shop speichert den Inhalt deines Warenkorbs.
4. Drittanbieter-Cookies (Third-Party Cookies):
   1. Werden von anderen Websites gesetzt (z. B. Werbenetzwerke wie Google Ads).
   2. Beispiel: Tracking von Nutzerverhalten über verschiedene Seiten hinweg.

Für uns stellt sich nun die Frage wie wir mit Cookies mit Hilfe von "requests" umgehen können. Wir können mit der httpbin API Cookies setzen:

```python
import requests
import json

BASE_URL = "https://httpbin.org/"
ENDPOINT_1 = "get"
ENDPOINT_2 = "redirect/3"
ENDPOINT_3 = "cookies/set?name=JohnDoe"

try:
    response = requests.get(BASE_URL + ENDPOINT_3)
    response.raise_for_status()

    data = response.json()
    print(json.dumps(data, indent=4))

except requests.exceptions.HTTPError as e:
    print(f"Fehler: {e}")
except json.JSONDecodeError:
    print("Fehler: Antwort ist kein gültiges JSON!")
```

Als Antwort bekommen wir:

```
{
    "cookies": {
        "name": "JohnDoe"
    }
}
```

Wir können im Web-Browser die URL https://httpbin.org/cookies/set?freeform=JohnDoe aufrufen und in den DevTools sehen, dass Cookies gesetzt wurden:

<img src="../img/requests_09.png" alt="Client-Server_Modell_01" width="550">

Folgender Ablauf ist eben passiert:
1. Die Anfrage wird an https://httpbin.org/cookies/set?name=JohnDoe gesendet.
2. Der Server antwortet mit einer JSON-Bestätigung, dass das Cookie gesetzt wurde.
3. Das Cookie wird jedoch nicht in response.cookies gespeichert, weil requests.get() keine Cookies speichert. Deswegen werden hier keine Cookies angezeigt:
```
Erhaltene Cookies: <RequestsCookieJar[]>
Cookies als Dictionary: {}
```

Bisher haben wir hauptsächlich "requests.get()" verwendet, um einzelne Anfragen zu senden. Funktionen wie "requests.get()" oder "requests.post()"  sind High-Level-APIs, die Details wie Verbindungsmanagement und Sitzungsverwaltung abstrahieren. Man kann mit einem sogenannten Session-Objekt mehr Kontrolle erhalten. Das Session-Objekt von requests bietet eine dauerhafte Verbindung zu einem Server. Es speichert Cookies, Header, Authentifizierungsdaten und wiederverwendet dieselbe TCP-Verbindung für mehrere Anfragen.
Wichtige Vorteile bie der Nutzung des Session-Objektes sind:
- Verbindung bleibt bestehen, sodass weniger Ressourcen verbraucht werden. Ohne Session muss jeder "requests.get()" Befehlt eine neue TCP-Verbindung erstellen, was langsamer ist.
- Automatische Cookie-Verwaltung, nützlich für Login-Sitzungen.
- Standard-Header und Parameter für alle Anfragen setzen.
- Wiederverwendbare Authentifizierung für APIs.

Einer der größten Vorteile von Session ist das Speichern von Cookies über mehrere Anfragen hinweg:

```python
import requests
import json

BASE_URL = "https://httpbin.org/"
ENDPOINT_1 = "get"
ENDPOINT_2 = "redirect/3"
ENDPOINT_3 = "cookies/set?name=JohnDoe"
ENDPOINT_4 = "cookies"

try:

    session = requests.Session()

    # Cookies setzen:
    response = session.get(BASE_URL + ENDPOINT_3)

    # Cookies abrufen:
    response = session.get(BASE_URL + ENDPOINT_4)
    response.raise_for_status()

    data = response.json()
    print(json.dumps(data, indent=4))

    cookies = session.cookies
    print(f"Erhaltene Cookies: {cookies}") # <RequestsCookieJar[]>

    cookies_dict = cookies.get_dict()
    print("Cookies als Dictionary:", cookies_dict)

except requests.exceptions.HTTPError as e:
    print(f"Fehler: {e}")
except json.JSONDecodeError:
    print("Fehler: Antwort ist kein gültiges JSON!")
```

Liefert als Ausgabe:

```
{
    "cookies": {
        "name": "JohnDoe"
    }
}
Erhaltene Cookies: <RequestsCookieJar[<Cookie name=JohnDoe for httpbin.org/>]>
Cookies als Dictionary: {'name': 'JohnDoe'}
```

Manchmal müssen wir selbst Cookies setzen, wenn der Server keine Cookies sendet oder wir sie vorgeben möchten:

```python
import requests
import json

BASE_URL = "https://httpbin.org/"
ENDPOINT_1 = "get"
ENDPOINT_2 = "redirect/3"
ENDPOINT_3 = "cookies/set?name=JohnDoe"
ENDPOINT_4 = "cookies"

try:

    session = requests.Session()

    # Cookies setzen:
    session.cookies.set("user", "JohnDoe")
    session.cookies.set("theme", "dark")

    # Cookies abrufen:
    response = session.get(BASE_URL + ENDPOINT_4)
    response.raise_for_status()

    data = response.json()
    print(json.dumps(data, indent=4))

    cookies = session.cookies
    print(f"Erhaltene Cookies: {cookies}") # <RequestsCookieJar[]>

    cookies_dict = cookies.get_dict()
    print("Cookies als Dictionary:", cookies_dict)

except requests.exceptions.HTTPError as e:
    print(f"Fehler: {e}")
except json.JSONDecodeError:
    print("Fehler: Antwort ist kein gültiges JSON!")
```

Schauen wir uns ein Anwendungsbeispiel im Bezug zu Cookies an. Wir simulieren einen Fake-Login, speichern eine Session-ID als Cookie und zeigen, wie man sie nutzt, um Benutzerdaten abzurufen. Wie funktioniert diese Anwendung?
1. Login:
   1. Wenn man auf "Login" klickt, sendet die App eine GET-Anfrage an https://httpbin.org/cookies/set/sessionID/abcdef123456.
   2. Die Antwort enthält ein Cookie (sessionID), das im requests.Session()-Objekt gespeichert wird.
   3. Falls das Cookie gespeichert wurde, ändert sich der Status auf "Du bist eingeloggt!".
2. Benutzerdaten abrufen:
   1. Beim Klick auf "Benutzerdaten abrufen" ruft die App die gespeicherten Cookies mit https://httpbin.org/cookies ab.
   2. Das zeigt, welche Daten der Server aus der Sitzung kennt.
3. Logout (beim Klick auf "Logout"):
   1. Sendet die App eine Anfrage an https://httpbin.org/cookies/delete?sessionID, um das Cookie zu löschen.
   2. Danach wird die Session geleert und der Status ändert sich auf "Nicht mehr eingeloggt!".
   
```python
import requests

# Basis-URL von httpbin.org
BASE_URL = "https://httpbin.org/"

LOGIN_ENDPOINT = "cookies/set/sessionID/abcdef123456"
USER_DATA_ENDPOINT = "cookies"
LOGOUT_ENDPOINT = "cookies/delete?sessionID"

session = requests.Session()

def login():
    """Simuliert einen Login und speichert das Session-Cookie."""
    response = session.get(BASE_URL + LOGIN_ENDPOINT)
    
    # Prüfe, ob das Session-Cookie erfolgreich gespeichert wurde
    if "sessionID" in session.cookies:
        print("✅ Login erfolgreich! Session-ID gespeichert:", session.cookies.get("sessionID"))
    else:
        print("Login fehlgeschlagen! Kein Session-Cookie erhalten.")

def get_user_data():
    """Ruft die gespeicherten Cookies vom Server ab und zeigt sie an."""
    response = session.get(BASE_URL + USER_DATA_ENDPOINT)
    
    if response.status_code == 200:
        print("Server kennt folgende Cookies:")
        print(response.json())
    else:
        print("Fehler beim Abrufen der Benutzerdaten!")

def logout():
    """Löscht die Session-ID und setzt die Session zurück."""
    session.get(BASE_URL + LOGOUT_ENDPOINT)

    # Lösche alle gespeicherten Cookies
    session.cookies.clear()

    print("Erfolgreich ausgeloggt! Alle Cookies wurden entfernt.")

if __name__ == "__main__":
    print("\n--- Starte Anwendung ---\n")

    # 1. Login
    print("1. Login")
    login()

    # 2. Benutzerdaten abrufen
    print("\n2. Benutzerdaten abrufen:")
    get_user_data()

    # 3. Logout
    print("\n3. Logout:")
    logout()

    # 4. Prüfen, ob wirklich ausgeloggt wurde
    print("\n4. Benutzerdaten nach Logout abrufen:")
    get_user_data()
```